# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.7.4 ランダムフォレスト (Random Forests, RF)
### クラス分類

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

### あやめのデータセット (iris_with_species.csv)
有名な [Fisher’s Iris Data](https://en.wikipedia.org/wiki/Iris_flower_data_set)。150個のあやめについて、がく片長(Sepal Length)、がく片幅(Sepal Width)、花びら長(Petal Length)、花びら幅(Petal Width)が計測されています。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0, header=0) # あやめのデータセットの読み込み

RF は DT と同様にして、3 つのクラスがあっても問題ありません。setosa, versicolor, virginica の 3 つのクラスを分類します。

In [ ]:
# y と x に分割
y = dataset.iloc[:,0]
x = dataset.iloc[:,1:]

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=50, stratify=y, shuffle=True, random_state=3)

DT モデルと同様にして RF モデルにおいても、一般的に x の標準化 (オートスケーリング) は行いません。

RF の実行

In [ ]:
from sklearn.ensemble import RandomForestClassifier # クラス分類用の RF の実行に使用

In [ ]:
model = RandomForestClassifier(n_estimators=500, max_features=0.5, oob_score=True) # RFモデルの宣言

In [ ]:
model.fit(x_train, y_train) # DTモデル構築

構築された RF モデルにおける説明変数 x の重要度

In [ ]:
model.feature_importances_ # 特徴量の重要度。array 型で出力されます

In [ ]:
importances = pd.DataFrame(model.feature_importances_) # pandas の DataFrame 型に変換

In [ ]:
importances.index = x_train.columns # 説明変数に対応する名前を、元のデータの説明変数名に

In [ ]:
importances.columns = ['importances'] # 列名を変更

In [ ]:
importances # 念のため確認

In [ ]:
importances.to_csv('importances.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

Out Of Bag (OOB) における正解率

In [ ]:
model.oob_score_

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(x_train), index=x_train.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics # 混同行列の作成、正解率の計算に使用

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(x_test), index=x_test.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

OOB を用いた説明変数 x の割合の最適化

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
ratios_of_x = np.arange(0.1, 1.1, 0.1) # 用いる説明変数の割合の候補

In [ ]:
ratios_of_x # 念のため確認

In [ ]:
accuracy_oob = [] # 空の list。説明変数の数の割合ごとに、OOB における正解率を入れていきます

In [ ]:
for ratio_of_x in ratios_of_x:
    model = RandomForestClassifier(n_estimators=500, max_features=ratio_of_x, oob_score=True, random_state=1)
    model.fit(x_train, y_train)
    accuracy_oob.append(model.oob_score_)

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
# 結果の確認
plt.rcParams['font.size'] = 18
plt.scatter(ratios_of_x, accuracy_oob)
plt.xlabel('ratio of x')
plt.ylabel('accuracy for OOB')
plt.show()

In [ ]:
optimal_ratio_of_x = ratios_of_x[accuracy_oob.index(max(accuracy_oob))] # OOB における正解率が最大となる選択する x の割合

In [ ]:
optimal_ratio_of_x # 念のため確認

RF モデルの構築および予測

In [ ]:
model = RandomForestClassifier(n_estimators=500, max_features=optimal_ratio_of_x, oob_score=True) # RFモデルの宣言

In [ ]:
model.fit(x_train, y_train) # RF モデル構築

構築された RF モデルにおける説明変数 x の重要度

In [ ]:
model.feature_importances_ # 特徴量の重要度。array 型で出力されます

In [ ]:
importances = pd.DataFrame(model.feature_importances_) # pandas の DataFrame 型に変換

In [ ]:
importances.index = x_train.columns # 説明変数に対応する名前を、元のデータの説明変数名に

In [ ]:
importances.columns = ['importances'] # 列名を変更

In [ ]:
importances # 念のため確認

In [ ]:
importances.to_csv('importances.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

Out Of Bag (OOB) における正解率

In [ ]:
model.oob_score_

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(x_train), index=x_train.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics # 混同行列の作成、正解率の計算に使用

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(x_test), index=x_test.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。